In [ ]:
import pandas as pd
import os

### Notes

Trying to experiment with loading in data programmatically

In [ ]:
files = os.listdir('repo_data/')

In [ ]:
initial_dfs = []
for dir, subdir, files in os.walk('repo_data/'):
    print("directory", dir)
    # print("subdirectory", subdir)
    # print("files", files)
    for file in files:
        if file.endswith('.csv'):
            # print("file", file)
            df = pd.read_csv(os.path.join(dir, file))
            df['file_path'] = os.path.join(dir, file)
            df['directory'] = dir
            initial_dfs.append(df)

In [ ]:
combined_df = pd.concat(initial_dfs)

In [ ]:
combined_df.directory.value_counts()

- Need to subset the data to only include repositories related to teaching

In [ ]:
combined_df

In [ ]:
potential_pedagogy_terms = [
    "pedagogy",
    "teaching",
    "education",
    "learning",
    "curriculum",
    "instruction",
    "syllabus",
    "paper",
    "class",
    "course",
    "lesson",
    "lecture",
    "semester"
    ]

### lower casing description and names, loop through each term

In [ ]:
combined_df["lower_description"] = combined_df["description"].str.lower()
combined_df["lower_name"] = combined_df["name"].str.lower()


In [ ]:
dfs_combined = []
for term in potential_pedagogy_terms:
    selected_rows = combined_df[(combined_df["lower_description"].str.contains(term)) | (combined_df["lower_name"].str.contains(term))]
    print(term, len(selected_rows))
    dfs_combined.append(selected_rows)

In [ ]:
combined_pedagogy_df = pd.concat(dfs_combined)
combined_pedagogy_df = combined_pedagogy_df.drop_duplicates(subset=['id'])
combined_pedagogy_df

In [ ]:
combined_pedagogy_df.to_csv('combined_pedagogy_data.csv',index=False)

# Im not sure this df was read into the csv in the way we want (?)
# Im understanding the dataframe better than the csv

### Larger Research Questions / Aims and Next Coding Steps
- Data Collected !  {combined_pedagogy_data.csv})
- Skills Analyzed   (skills being taught to students, return to Hack/Tact/Yack, do a Hack/Tact/Yack word count analysis of Dataset 1)
- Pedagogy Analyzed  (Pedagogical Practices Considered, do a Pedagogy Approaches word count analysis of the combined pedagogy data) 
- Present Current Practices, what skills are most taught, what pedagogical approaches most used
- Best Practices Evaluation: what are instructor / pedagogical goals? what skills are most effective at achieving instructor goals? What pedagogical approaches are most effective in students achieving certain skills and/or reaching pedagogical goals? 
    - value of interviews here, qualitative questions about big picture goals (not just "i want my dh students to learn coding/python" but also "i want them to think algorithmically / understand affordances and limitations of digital methods / do x kind of research..." )
    - similar qualitative research / questions about if students were able to meet these goals and through what skills and what pedagogical approaches (ie learning python loops helped and so did group projects)
    - maybe we want to limit questions to success of just teaching concrete skills through which approaches (ie coding + projects, not history of the field through group reflection posting)
- Owen Next Step ? --> Hack Tact Yack word count breakdowns for repo_data and Pedagogy Approach word breakdown for combined_pedagogy data_csv or df



## Hack Word Count within pedagogy data 

In [ ]:
potential_hack_terms = [
    'construct',
    'create',
    'stand up',
    'web application',
    'interface',
    'web design',
    'coding',
    'computation',
    'programming',
    'algorithm',
    'processing',
    'machine learning',
    'software development',
    'data analysis',
    'text analysis',
    'network analysis',
    'archive',
    'dataset',
    'maps',
    'visualization',
    'techniques',
    'python',
    'django',
    'omeka',
    'GitHub',
    'Safari Techbooks',
    'Developer Networks',
    'Programming Environment',
    'Mozilla',
    'Anaconda',
    'GIS',
    'Visual Studio Code',
    'TEI',
    'SQL',
    ]

In [ ]:
combined_pedagogy_repo_data = pd.read_csv(
    "combined_pedagogy_data.csv")

In [ ]:
dfs_hack = []
for term in potential_hack_terms:
    selected_rows = combined_pedagogy_repo_data[(combined_pedagogy_repo_data["lower_description"].str.contains(term)) | (combined_pedagogy_repo_data c["lower_name"].str.contains(term))]
    print(term, len(selected_rows))
    dfs_digital_history.append(selected_rows)